# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f5444c03f10>
├── 'a' --> tensor([[-1.5743,  0.3775, -1.1941],
│                   [ 1.3848,  0.9011, -0.8879]])
└── 'x' --> <FastTreeValue 0x7f5444c03df0>
    └── 'c' --> tensor([[ 0.0358, -1.6137, -1.4106, -0.9694],
                        [-0.9107, -0.7029, -1.3674,  0.7502],
                        [ 0.9555,  1.0081, -0.8387,  1.3931]])

In [4]:
t.a

tensor([[-1.5743,  0.3775, -1.1941],
        [ 1.3848,  0.9011, -0.8879]])

In [5]:
%timeit t.a

76.4 ns ± 3.56 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f5444c03f10>
├── 'a' --> tensor([[0.3871, 0.7161, 0.8446],
│                   [0.1139, 0.7433, 0.3537]])
└── 'x' --> <FastTreeValue 0x7f5444c03df0>
    └── 'c' --> tensor([[ 0.0358, -1.6137, -1.4106, -0.9694],
                        [-0.9107, -0.7029, -1.3674,  0.7502],
                        [ 0.9555,  1.0081, -0.8387,  1.3931]])

In [7]:
%timeit t.a = new_value

79.3 ns ± 2.4 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.5743,  0.3775, -1.1941],
               [ 1.3848,  0.9011, -0.8879]]),
    x: Batch(
           c: tensor([[ 0.0358, -1.6137, -1.4106, -0.9694],
                      [-0.9107, -0.7029, -1.3674,  0.7502],
                      [ 0.9555,  1.0081, -0.8387,  1.3931]]),
       ),
)

In [10]:
b.a

tensor([[-1.5743,  0.3775, -1.1941],
        [ 1.3848,  0.9011, -0.8879]])

In [11]:
%timeit b.a

64 ns ± 1.22 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.1246, -0.9731,  1.2495],
               [ 0.3110,  0.7596,  0.2190]]),
    x: Batch(
           c: tensor([[ 0.0358, -1.6137, -1.4106, -0.9694],
                      [-0.9107, -0.7029, -1.3674,  0.7502],
                      [ 0.9555,  1.0081, -0.8387,  1.3931]]),
       ),
)

In [13]:
%timeit b.a = new_value

566 ns ± 9.96 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

966 ns ± 27.1 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.1 µs ± 200 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

253 µs ± 8.02 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

255 µs ± 7.03 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f539a8a3a60>
├── 'a' --> tensor([[[-1.5743,  0.3775, -1.1941],
│                    [ 1.3848,  0.9011, -0.8879]],
│           
│                   [[-1.5743,  0.3775, -1.1941],
│                    [ 1.3848,  0.9011, -0.8879]],
│           
│                   [[-1.5743,  0.3775, -1.1941],
│                    [ 1.3848,  0.9011, -0.8879]],
│           
│                   [[-1.5743,  0.3775, -1.1941],
│                    [ 1.3848,  0.9011, -0.8879]],
│           
│                   [[-1.5743,  0.3775, -1.1941],
│                    [ 1.3848,  0.9011, -0.8879]],
│           
│                   [[-1.5743,  0.3775, -1.1941],
│                    [ 1.3848,  0.9011, -0.8879]],
│           
│                   [[-1.5743,  0.3775, -1.1941],
│                    [ 1.3848,  0.9011, -0.8879]],
│           
│                   [[-1.5743,  0.3775, -1.1941],
│                    [ 1.3848,  0.9011, -0.8879]]])
└── 'x' --> <FastTreeValue 0x7f539a8d3f40>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

40.8 µs ± 961 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f539a8d5dc0>
├── 'a' --> tensor([[-1.5743,  0.3775, -1.1941],
│                   [ 1.3848,  0.9011, -0.8879],
│                   [-1.5743,  0.3775, -1.1941],
│                   [ 1.3848,  0.9011, -0.8879],
│                   [-1.5743,  0.3775, -1.1941],
│                   [ 1.3848,  0.9011, -0.8879],
│                   [-1.5743,  0.3775, -1.1941],
│                   [ 1.3848,  0.9011, -0.8879],
│                   [-1.5743,  0.3775, -1.1941],
│                   [ 1.3848,  0.9011, -0.8879],
│                   [-1.5743,  0.3775, -1.1941],
│                   [ 1.3848,  0.9011, -0.8879],
│                   [-1.5743,  0.3775, -1.1941],
│                   [ 1.3848,  0.9011, -0.8879],
│                   [-1.5743,  0.3775, -1.1941],
│                   [ 1.3848,  0.9011, -0.8879]])
└── 'x' --> <FastTreeValue 0x7f539a8d5970>
    └── 'c' --> tensor([[ 0.0358, -1.6137, -1.4106, -0.9694],
                        [-0.9107, -0.7029, -1.3674,  0.7502],
                 

In [23]:
%timeit t_cat(trees)

54.9 µs ± 6.78 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

72.3 µs ± 1.92 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-1.5743,  0.3775, -1.1941],
                [ 1.3848,  0.9011, -0.8879]],
       
               [[-1.5743,  0.3775, -1.1941],
                [ 1.3848,  0.9011, -0.8879]],
       
               [[-1.5743,  0.3775, -1.1941],
                [ 1.3848,  0.9011, -0.8879]],
       
               [[-1.5743,  0.3775, -1.1941],
                [ 1.3848,  0.9011, -0.8879]],
       
               [[-1.5743,  0.3775, -1.1941],
                [ 1.3848,  0.9011, -0.8879]],
       
               [[-1.5743,  0.3775, -1.1941],
                [ 1.3848,  0.9011, -0.8879]],
       
               [[-1.5743,  0.3775, -1.1941],
                [ 1.3848,  0.9011, -0.8879]],
       
               [[-1.5743,  0.3775, -1.1941],
                [ 1.3848,  0.9011, -0.8879]]]),
    x: Batch(
           c: tensor([[[ 0.0358, -1.6137, -1.4106, -0.9694],
                       [-0.9107, -0.7029, -1.3674,  0.7502],
                       [ 0.9555,  1.0081, -0.8387,  1.3931]],
         

In [26]:
%timeit Batch.stack(batches)

94.7 µs ± 1.83 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-1.5743,  0.3775, -1.1941],
               [ 1.3848,  0.9011, -0.8879],
               [-1.5743,  0.3775, -1.1941],
               [ 1.3848,  0.9011, -0.8879],
               [-1.5743,  0.3775, -1.1941],
               [ 1.3848,  0.9011, -0.8879],
               [-1.5743,  0.3775, -1.1941],
               [ 1.3848,  0.9011, -0.8879],
               [-1.5743,  0.3775, -1.1941],
               [ 1.3848,  0.9011, -0.8879],
               [-1.5743,  0.3775, -1.1941],
               [ 1.3848,  0.9011, -0.8879],
               [-1.5743,  0.3775, -1.1941],
               [ 1.3848,  0.9011, -0.8879],
               [-1.5743,  0.3775, -1.1941],
               [ 1.3848,  0.9011, -0.8879]]),
    x: Batch(
           c: tensor([[ 0.0358, -1.6137, -1.4106, -0.9694],
                      [-0.9107, -0.7029, -1.3674,  0.7502],
                      [ 0.9555,  1.0081, -0.8387,  1.3931],
                      [ 0.0358, -1.6137, -1.4106, -0.9694],
                      [-0.9107, -

In [28]:
%timeit Batch.cat(batches)

169 µs ± 4.47 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

522 µs ± 4.01 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
